# Dog-Friendly Neighbourhoods of Stockholm

<img src="https://miro.medium.com/max/1800/1*Ajbb76yGEqKPRdbp-rrn4g.jpeg" alt="dogswelcome" align="left" width=600>

<p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Hello, My name is Liuba and I live in Gothenburg with my dog and sidekick called Watson. We live in a neighbourhood called Olskroken. When I was buying my appartment I chose this neighbourhood for it's dog-friendliness (before I even met Watson!).</p>
<p><strong>What does a dog-friendly neighbourhood mean?</strong> From my point of view a neighbourhood can be called dog-friendly if it has the following attributes:</p>
<ul>
<li>A forest or a park</li>
<li>An Animal Hospital</li>
<li>A Dog Park (Called "hundrasgarden" in Swedish)</li>
<li>Doggy Daycare (Called "hunddagis" in Swedish)</li>
<li>A Pet Store</li>
<li>A Pet Salon</li>
<li>Dog-friendly cafes and restaurants</li>
</ul>
<p>It is not necessary for a neighbourhood to have all the attributes mentioned above to be called dog-friendly but the more checkboxes it ticks the higher it would be on my list.</p>
<p><strong>Now to the problem and goal of this project</strong>: I am looking into moving to Stockholm and I would like to find a dog-friendly area to live in. The goal of this project would be to determine and compare dog-friendly neighbourhoods in Stockholm.</p>
<p>I believe the end results of this analysis would be beneficial to any dog owner living in Stockholm or someone who wants to move with their furry buddy to this city.</p>
<p>I will be using Foursquare API to retrieve venue information and any data that I can scrape on the&nbsp;Wikipedia or the internet about the neighbourhoods of Stockholm.</p>

# Step 1. Downloading the libraries that will be required

In [3]:
import pandas as pd # library for data analsysis
import numpy as np  # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library

print('Libraries imported.')

# Step 2. Processing the data

After a bit of search for map data about the districts of Stockholm, I ended up manually scrapping the Wikipedia page https://en.wikipedia.org/wiki/Districts_of_Sweden for information about the districts. I have created the file called stockholm_districts.csv containing the Boroughs and Districts of Stockholm.

In [8]:
data = pd.read_csv("/Users/liuba/Desktop/GitHub/Coursera_Capstone/stockholm_districts.csv")

Next I added the columns for lattitude and longitude of each district and I retrieved this information using Nominatim that we learned about during the labs.

In [9]:
for i in range(115):
    address = data.iloc[i,2] + ", "+ data.iloc[i,1] + ", "+ data.iloc[i,0]
    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    if location is not None:
        data.at[i,'Lat'] = float(location.latitude)
        data.at[i,'Long'] = float(location.longitude)
        #print(i,float(location.latitude)," ",float(location.longitude))
    else:
        address = data.iloc[i,2] + ", "+ data.iloc[i,0]
        geolocator = Nominatim(user_agent="explorer")
        location = geolocator.geocode(address)
        if location is not None:
            data.at[i,'Lat'] = float(location.latitude)
            data.at[i,'Long'] = float(location.longitude)
            #print(i,float(location.latitude)," ",float(location.longitude))
        else:
            data.at[i,'Lat'] = float(0)
            data.at[i,'Long'] = float(0)
            #print(i,float(0)," ",float(0))

In [10]:
#Checking if there is any address that was not located.
data.loc[(data['Lat'] == float(0))]

Next I proceed to create the map of Stockholm and I added markers for the districts as well.

In [11]:
address = 'Stockholm,Sweden'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm are {}, {}.'.format(latitude, longitude))

In [12]:
# Creating a map of Stockholm using latitude and longitude values
map_stkhlm = folium.Map(location=[latitude, longitude], zoom_start=11)

# Adding markers of the districts to the map
for lat, lng, borough, district in zip(data['Lat'], data['Long'], data['Borough'], data['District']):
    label = '{}, {}'.format(district, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stkhlm)  
    
map_stkhlm

I noticed that a couple of districts where mapped wrong so I manually corrected their coordinates.

In [13]:
data.at[15,'Lat'] = float(59.390159)
data.at[15,'Long'] = float(17.872202)
data.at[95,'Lat'] = float(59.251924)
data.at[95,'Long'] = float(18.174457)

In [14]:
# Creating a map of Stockholm using latitude and longitude values
map_stkhlm = folium.Map(location=[latitude, longitude], zoom_start=11)

# Adding markers of the districts to the map
for lat, lng, borough, district in zip(data['Lat'], data['Long'], data['Borough'], data['District']):
    label = '{}, {}'.format(district, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stkhlm)  
    
map_stkhlm

Much better! I am not sure if I retrieved ALL Stockholm districts and whether all the markers are placed correctly but I believe at this point we have enough data to start exploring the districts with the help of Foursquare.

Before I did that, I saved the processed data into a cvs format to be able to retrieve it without the need to process it again.

In [15]:
data.to_csv(r'/Users/liuba/Desktop/GitHub/Coursera_Capstone/stockholm_districts_coords.csv', index=False)

In [16]:
data = pd.read_csv("/Users/liuba/Desktop/GitHub/Coursera_Capstone/stockholm_districts_coords.csv")
data.head()

# Step 3. Utilizing the Foursquare API to explore the neighborhoods and segment them

<p>First I examined the Foursquare venue categories (<a href="https://developer.foursquare.com/docs/resources/categories">https://developer.foursquare.com/docs/resources/categories</a>) to determine which ones will be relevant for the goal of this analysis.&nbsp;</p>
<p>I think the following categories will by the key features for dog-friendly districts.&nbsp;</p>
<ul>
<li><strong>Pet Caf&eacute;</strong> 56aa371be4b08b9a8d573508</li>
<li><strong>Dog Run</strong> 4bf58dd8d48988d1e5941735</li>
<li><strong>Park</strong> 4bf58dd8d48988d163941735</li>
<li><strong>Trail</strong> 4bf58dd8d48988d159941735</li>
<li><strong>Veterinarian</strong> 4d954af4a243a5684765b473</li>
<li><strong>Pet Service</strong> 5032897c91d4c4b30a586d69</li>
<li><strong>Pet Store</strong> 4bf58dd8d48988d100951735</li>
</ul>

In [17]:
config = json.load(open('/Users/liuba/Desktop/GitHub/Coursera_Capstone/config.json'))
CLIENT_ID = config['CLIENT_ID']
CLIENT_SECRET = config['CLIENT_SECRET']
VERSION = config['VERSION']

In [43]:
# '4bf58dd8d48988d163941735', '4bf58dd8d48988d159941735',
category_ids = ['56aa371be4b08b9a8d573508', '4bf58dd8d48988d1e5941735',  '4d954af4a243a5684765b473', '5032897c91d4c4b30a586d69', '4bf58dd8d48988d100951735']
category_ids = ",".join(category_ids)
radius = 20000
LIMIT = 100

In [44]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_ids, radius, LIMIT)
url
results = requests.get(url).json()
results

d8d48988d100951735',
      'name': 'Pet Store',
      'pluralName': 'Pet Stores',
      'shortName': 'Pet Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1570880920',
    'hasPerk': False},
   {'id': '4bd83cdf35aad13af45a90f3',
    'name': 'Evidensia Djurkliniken Roslagstull',
    'location': {'address': 'Valhallavägen 12a',
     'lat': 59.3492001,
     'lng': 18.0608958,
     'labeledLatLngs': [{'label': 'display',
       'lat': 59.3492001,
       'lng': 18.0608958}],
     'distance': 2742,
     'postalCode': '114 22',
     'cc': 'SE',
     'city': 'Stockholm',
     'state': 'Storstockholm',
     'country': 'Sverige',
     'formattedAddress': ['Valhallavägen 12a', '114 22 Stockholm', 'Sverige']},
    'categories': [{'id': '4d954af4a243a5684765b473',
      'name': 'Veterinarian',
      'pluralName': 'Veterinarians',
      'shortName': 'Veterinarians',
      'icon': {'pref

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [37]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [38]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Stockholm

# add the dog-friendly place as red circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [62]:
# Creating a function to extract venues information of a neighbourhood in Toronto
radius = 500
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, category_ids, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
       
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venues']['name'], 
            v['venues']['location']['lat'], 
            v['venues']['location']['lng'],  
            v['venues']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [76]:
radius = 500
LIMIT=100
name = data.at[6,'District']
lat = data.at[6,'Lat']
lng = data.at[6,'Long']
venues_list = []

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, category_ids, radius, LIMIT)
            
# make the GET request
results = requests.get(url).json()['response']

# return only relevant information for each nearby venue
venues_list.append([(
    name, 
    lat, 
    lng, 
    v['venues']['name'], 
    v['venues']['location']['lat'], 
    v['venues']['location']['lng'],  
    v['venues']['categories']['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['District', 
                'District Latitude', 
                'District Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']

TypeError: string indices must be integers

In [65]:
stckhlm_venues = getNearbyVenues(names=data['Borough'],
                                   latitudes=data['Lat'],
                                   longitudes=data['Long']
                                  )

Älvsjö
Älvsjö
Älvsjö
Älvsjö
Älvsjö
Älvsjö


KeyError: 'venues'

In [ ]:
print(stckhlm_venues.shape)
stckhlm_venues.head()

In [66]:
data

,City,Borough,District,Lat,Long
0,Stockholm,Älvsjö,Herrängen,59.273428,17.964587
1,Stockholm,Älvsjö,Långsjö,59.267506,17.978904
2,Stockholm,Älvsjö,Långbro,59.282433,17.982491
3,Stockholm,Älvsjö,Älvsjö,59.275849,18.001889
4,Stockholm,Älvsjö,Solberga,59.279574,18.007000
5,Stockholm,Älvsjö,Örby Slott,59.280940,18.029227
6,Stockholm,Älvsjö,Liseberg,59.286121,18.026484
7,Stockholm,Bromma,Abrahamsberg,59.332459,17.949192
8,Stockholm,Bromma,Alvik,59.333401,17.982279
9,Stockholm,Bromma,Beckomberga,59.358219,17.904932
